In [ ]:
import json
import requests

def inference(url, prompt):
    payload = {
        "model": 'deepseek-r1:1.5b',
        "prompt": prompt,
        "temperature": 0.5,
        "max_tokens": 2
    
    }
    # Send the request
    response = requests.post(url, json=payload)
    final_response = ""
    for chunk in response.text.split("\n"):
        if chunk.strip():  # Skip empty lines
            data = json.loads(chunk)
            final_response += data.get("response", "")
    return final_response


if __name__=='__main__':
    # Define the request parameters
    url = "http://localhost:11434/api/generate"
    t = True
    while t==True:
        prompt = input("Enter your query here: ")
        if prompt=="exit":
            t = False
        else:
            final_response = inference(url, prompt)
            print(final_response)

In [ ]:
import json
import requests

def inference(url, prompt):
    payload = {
        "model": 'deepseek-r1:1.5b',
        "prompt": prompt,
        "temperature": 0.5,
        "max_tokens": 100  # Increase if you want longer responses
    }
    # Send the request
    response = requests.post(url, json=payload)
    final_response = ""
    for chunk in response.text.split("\n"):
        if chunk.strip():  # Skip empty lines
            data = json.loads(chunk)
            final_response += data.get("response", "")
    return final_response

if __name__ == '__main__':
    url = "http://localhost:11434/api/generate"
    print("Type 'exit' or 'quit' to stop the chatbot.\n")
    
    while True:
        prompt = input("You: ")
        if prompt.strip().lower() in ['exit', 'quit']:
            print("Chatbot: Goodbye!")
            break
        try:
            final_response = inference(url, prompt)
            print("Chatbot:", final_response)
        except Exception as e:
            print("Error:", e)

Type 'exit' or 'quit' to stop the chatbot.



In [3]:
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
import re
# Step 1: Load documents (example corpus)
documents = [
    "Python is a popular programming language for AI.",
    "Threading in Python allows concurrent execution of code.",
    "Llama 3.2 is a state-of-the-art language model by Ollama.",
    "RAG stands for Retrieval-Augmented Generation.",
    "RAG is very popular in making chatbot"
  
]

# Step 2: Convert documents to TF-IDF vectors
vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(documents)

# Function to retrieve relevant documents
def retrieve_docs(query):
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, doc_vectors)
    most_similar_idx = similarities.argmax()
    return documents[most_similar_idx]

# Step 3: Query the RAG system
def rag_query(query):
    relevant_doc = retrieve_docs(query)
    print(f"Retrieved Document: {relevant_doc}")
    
    # Step 4: Send the relevant document and query to Ollama API (using Llama 3.2)
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": 'deepseek-r1:1.5b', "prompt": f"{relevant_doc}\n\nQuery: {query}"}
    )
   
      # Handle the raw response text
        # Handle the raw response text
    raw_response = response.text
     # Extract all 'response' values from the JSON objects
    responses = re.findall(r'"response":\s*"([^"]+)"', raw_response)
    # Combine all parts into a complete sentence
    final_response = "".join(responses)
    print(f"Llama Response: {final_response}")

# Example Query
rag_query("What is RAG?")


Retrieved Document: RAG is very popular in making chatbot
Llama Response: \u003cthink\u003e\nOkay, so I need to figure out what RAG stands for and explain it. From the query, I remember that someone asked about RAG being very popular in making chatbots, and now I'm supposed to provide a detailed explanation.\n\nFirst, I should start by understanding what a chatbot is. A chatbot is an automated system designed to interact with a user through text or voice messages. It can generate responses, answer questions, and provide information on various topics.\n\nNow, RAG seems to be associated with chatbots but sounds more like something specific. The query mentions it's very popular in making chatbots, so I'm thinking of its name because of the \A\ at the end, which makes me wonder if it stands for something related to AI or human-computer interaction.\n\nWait, RAG—maybe it's an acronym? Let me think... RG is a common abbreviation. Maybe A is another part. If I break it down: R is robot, G cou

In [ ]:
# 1. Install required packages
# pip install pypdf langchain sentence-transformers faiss-cpu ollama

import ollama
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.schema import Document  # Add this import
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 2. Load and process PDF document
def load_pdf_documents(pdf_path):
    loader = PyPDFLoader(pdf_path)
    return loader.load()  # Returns list of Document objects

# 3. Split text into chunks
def split_documents(pages):  # Changed parameter name
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    return text_splitter.split_documents(pages)  # Directly split Document objects

# 4. Create vector store
def create_vector_store(split_docs):  # Changed parameter name
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    document_texts = [doc.page_content for doc in split_docs]  # Use .page_content
    embeddings = embedder.encode(document_texts)
    
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings.astype(np.float32))
    return index, document_texts, embedder

# 5. Retrieve relevant context (unchanged)
def retrieve_context(query, embedder, index, documents, k=3):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding.astype(np.float32), k)
    return [documents[i] for i in indices[0]]

# 6. Generate answer using Ollama (unchanged)
def generate_answer_with_ollama(query, context):
    formatted_context = "\n".join(context)
    
    prompt = f"""You are an expert assistant trained on document information.
    Use this context to answer the question:
    
    {formatted_context}
    
    Question: {query}
    
    Answer in detail using only the provided context:"""
    
    response = ollama.generate(
        model='deepseek-r1:1.5b',
        prompt= prompt,
        options={
            'temperature': 0.3,
            'max_tokens': 2000
        }
    )
    return response['response']

# Main workflow (modified)
def main(pdf_path, query):
    # Load and process PDF
    pages = load_pdf_documents(pdf_path)  # Get Document objects
    split_docs = split_documents(pages)   # Split properly
    
    # Create vector store
    index, document_texts, embedder = create_vector_store(split_docs)
    
    # Retrieve context
    context = retrieve_context(query, embedder, index, document_texts)
    
    # Generate answer
    answer = generate_answer_with_ollama(query, context)
    return answer

# Example usage (unchanged)
if __name__ == "__main__":
    pdf_path = r"C:\Users\Ajeet\Downloads\attention-is-all-you-need.pdf"

    query = input("Type the query: ")
    
    result = main(pdf_path, query)
    print("Answer:", result)

c:\Users\Ajeet\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: module 'faiss' has no attribute 'IndexFlatL2'